## Backpropagation multilayered Neural Network

In [1]:
import numpy as np
import pandas as pd
from backpropagation import BackPropagation


### Create training set from diagnosis.xlsx

In [2]:
data = pd.read_excel('diagnosis.xlsx')
data.head()
training_set = []
for index, row in data.iterrows():
    features = [row['Fever'], row['Cough'], row['Headache'], row['Tiredness'], row['Night Sweat']]
    result = [row['Pneumonia'], row['Flu'], row['Cold']]
    training_set.append((features, result))

### Initialize and learn multilayered neural network

In [3]:
bpnn = BackPropagation(training_set,[5, 5, 4, 3])
bpnn.backpropagation()
for row in training_set:
    net_input = row[0]
    bpnn.feed_forward(net_input)
    net_output = bpnn.output_activation
    print("Input: ", net_input, " Output: ", net_output)


Input:  [0.0, 0.5, 0.5, 0.0, 0.0]  Output:  [[0.03699057]
 [0.05997777]
 [0.92923934]]
Input:  [1.0, 1.0, 1.0, 1.0, 0.0]  Output:  [[0.1289983 ]
 [0.8717239 ]
 [0.04293265]]
Input:  [0.5, 1.0, 0.0, 1.0, 1.0]  Output:  [[0.84982941]
 [0.15629466]
 [0.02594552]]
Input:  [0.0, 0.5, 0.0, 0.0, 0.0]  Output:  [[0.05550391]
 [0.01709165]
 [0.95907086]]
Input:  [1.0, 0.5, 1.0, 0.5, 0.0]  Output:  [[0.11546453]
 [0.85843758]
 [0.06404405]]
Input:  [1.0, 1.0, 0.0, 0.5, 0.5]  Output:  [[0.93274238]
 [0.06053893]
 [0.02141237]]
